In [1]:
import sys
import time

sys.path.append("..")  # make sure we can run this from the repo
from IPython.display import Markdown, clear_output, display

import cohere

## Client example

In [2]:
co = cohere.Client()
start_time = time.time()
streaming_gens = co.generate(prompt="Hey! Don't worry, 🐝 happy~", max_tokens=20, stream=True)
for i, token in enumerate(streaming_gens):
    print(f"[{i}][{time.time()-start_time:.2f}s] {token}")
# the request id is available after the first token has streamed, and response so far is in texts
streaming_gens.ids, streaming_gens.texts

[0][0.45s] StreamingText(id='4c2fc597-7dbe-448d-b0ce-f3c7778fd6c7', index=0, text=' The')
[1][0.45s] StreamingText(id=None, index=0, text=' queen')
[2][0.45s] StreamingText(id=None, index=0, text=' bee')
[3][0.45s] StreamingText(id=None, index=0, text=' is')
[4][0.47s] StreamingText(id=None, index=0, text=' not')
[5][0.50s] StreamingText(id=None, index=0, text=' a')
[6][0.53s] StreamingText(id=None, index=0, text=' gentle')
[7][0.56s] StreamingText(id=None, index=0, text=' flower')
[8][0.59s] StreamingText(id=None, index=0, text=',')
[9][0.61s] StreamingText(id=None, index=0, text=' she')
[10][0.64s] StreamingText(id=None, index=0, text=' has')
[11][0.67s] StreamingText(id=None, index=0, text=' a')
[12][0.70s] StreamingText(id=None, index=0, text=' large')
[13][0.72s] StreamingText(id=None, index=0, text=' body')
[14][0.75s] StreamingText(id=None, index=0, text=',')
[15][0.78s] StreamingText(id=None, index=0, text=' a')
[16][0.80s] StreamingText(id=None, index=0, text=' terrible')
[17]

(['4c2fc597-7dbe-448d-b0ce-f3c7778fd6c7'],
 [' The queen bee is not a gentle flower, she has a large body, a terrible temperament,'])

## AsyncClient streaming example

In [3]:
prompt = "안녕하"
num_generations = 5
max_tokens = 50
texts = [f"**Generation #{i+1}:** {prompt}" for i in range(num_generations)]
displays = [display(display_id=True) for t in texts]

async with cohere.AsyncClient() as aio_co:
    start_time = time.time()
    aio_streaming_gens = await aio_co.generate(
        prompt=prompt,
        max_tokens=max_tokens,
        model="command-xlarge-nightly",
        num_generations=num_generations,
        stream=True,
    )

    async for token in aio_streaming_gens:
        texts[token.index] += token.text
        displays[token.index].update(Markdown(texts[token.index]))

**Generation #1:** 안녕하세요. 반갑습니다.
A: Hi. Nice to meet you.
B: Nice to meet you too.

A: What's your name?
B: My name is Brian.

**Generation #2:** 안녕하세요 페이지 자주 사용하는데 제가 가장 좋아하는 분이 되는 웹 사이트

**Generation #3:** 안녕하세요. 제 이름은 베트남인입니다. 영어로 말하는 것이 힘들이네요. 그래도

**Generation #4:** 안녕하세요! 한국에서는 이런 것들이 많이 부피하지만 나라 사는 사람들은 이것들을

**Generation #5:** 안녕하세요 안녕하세요 오늘은 두번째 영화의 부화입니다 영화의 상